In [1]:
import numpy as np
from numba import jit
import time
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from scipy.stats import norm

In [2]:
def Asian_Analytical(T, K, r, S0, sigma, N):
    
    '''
    Calculate Asian Option 
    Analytical expression for Asian call option based on geometric averages
    '''
    sigma_tilde = sigma * np.sqrt((2*N + 1)/(6*(N+1)))
    r_tilde = ((r - 0.5*sigma*sigma) + (sigma_tilde*sigma_tilde))/2
    d1 = (np.log(S0/K) + (r_tilde + 0.5*sigma_tilde**2)*T) /(np.sqrt(T)*sigma_tilde)
    d2 = (np.log(S0/K) + (r_tilde - 0.5*sigma_tilde**2)*T) /(np.sqrt(T)*sigma_tilde)
    
    
    return np.exp(-r*T) * (S0 * np.exp(r_tilde *T)*norm.cdf(d1) - K*norm.cdf(d2))

In [17]:
True_Asian_geo = Asian_Analytical(1, 99, 0.06, 100, 0.2, 1000)
print(True_Asian_geo)

6.333989180142548


In [4]:
@jit(nopython=False, fastmath = True)
def GBM_Euler(T, K, S, sigma, r, M):
    '''
    Inputs: Time, Strike price, asset price, vol, interest rate, number of steps
    '''
    
    dt = T/M
    S_all = np.zeros(M)
    S_all[0] = S
    
    for i in range(1, M):
        S_all[i] = S_all[i-1] + r* S_all[i-1] * dt + sigma * S_all[i-1] * np.sqrt(dt) * np.random.normal()
    return S_all

In [32]:
T = 1; K = 99; r = 0.06; S = 100; sigma = 0.2; trials = 10000; M = 1000; N=250

In [33]:
#T = 1; K = 29; r = 0.08; S = 30; sigma = 0.3; trials = 1000; M = 100; N=250

In [34]:
# https://stackoverflow.com/questions/43099542/python-easy-way-to-do-geometric-mean-in-python
def geo_mean_overflow(iterable):
    return  np.exp(np.log(iterable).mean())

In [35]:
def Control_Variate_Arith_Asian(T, K, r, S, sigma, trials, M, N, returnall=False):
    s_geo = np.zeros(trials)
    s_ar = np.zeros(trials)
    True_Asian_geo = Asian_Analytical(T, K, r, S, sigma, N)
    print(f"True asian {True_Asian_geo}")
    
    '''
    for i in range(trials):
        test = GBM_Euler(T, K, S, sigma, r, M)
        s_geo[i] = np.max((0, geo_mean_overflow(test) - K))
        
        S_arith = ((1/len(test)) * np.sum(test))
        s_ar[i] = np.max((0, S_arith - K))
    '''
    
    for i in range(trials ):
        test = GBM_Euler(T, K, S, sigma, r, M)
    #    s[i] = np.max((0, (test.prod())**(1/(len(test))) - K ))
        s_geo[i] = np.max((0, geo_mean_overflow(test)   - K))
 
    for i in range(trials):
        test = GBM_Euler(T, K, S, sigma, r, M)
        S_arith = ((1/len(test)) * np.sum(test))
        s_ar[i] = np.max((0, S_arith - K))


    V_MC_geo = np.mean(s_geo)
    var_geo = np.var(s_geo)
    print(f"Asian MC geometric: {V_MC_geo}")
    print(f"Asian MC geometric var: {var_geo}")
    CI_geo = [np.mean(s_geo) - 1.96*np.std(s_geo)/np.sqrt(len(s_geo)), np.mean(s_geo) + 1.96*np.std(s_geo)/np.sqrt(len(s_geo))]
    
    V_MC_arith = np.mean(s_ar)
    var_arith = np.var(s_ar)
    CI_arith = [np.mean(s_ar) - 1.96*np.std(s_ar)/np.sqrt(len(s_ar)), np.mean(s_ar) + 1.96*np.std(s_ar)/np.sqrt(len(s_ar))]
    
    print(f"Asian MC arith: {V_MC_arith}")
    print(f"Asian MC arith var: {var_arith}")
      
    # Compute optimal beta coefficient which minimizes the variance
    beta = (np.std(s_ar) / np.std(s_geo)) * np.corrcoef(s_ar, s_geo)[1][0]

    # Compute Control Variate estimate of arithmetic Asian call option
    arith_CV_est = V_MC_arith - beta*(V_MC_geo - True_Asian_geo)
        
    
    # Compute variance of CV estimate 
    var_CV_arith = var_arith - ((np.cov((s_ar, s_geo))[1][0]**2 ) / var_geo)

    print(f"beta: {beta}")
    print(f"CV est {arith_CV_est}")
    
    
    if returnall:  
        return arith_CV_est, var_CV_arith, V_MC_geo, var_geo, V_MC_arith, var_arith
    return arith_CV_est, var_CV_arith


In [36]:
arith_CV_est, var_CV_arith = Control_Variate_Arith_Asian(T, K, r, S, sigma, trials, M, N, returnall=False)
print(f"returned est: {arith_CV_est} returned var {var_CV_arith}")
#arith_CV_est, var_CV_arith, V_MC_geo, var_geo, V_MC_arith, var_arith = Control_Variate_Arith_Asian(T, K, r, S, sigma, trials, M, N, returnall=True)
#print(arith_CV_est, var_CV_arith, V_MC_geo, var_geo, V_MC_arith, var_arith)

True asian 6.330266777876057
Asian MC geometric: 6.843712280319653
Asian MC geometric var: 74.58809896637793
Asian MC arith: 6.9186454460956375
Asian MC arith var: 77.45183486862345
beta: 0.009402266704449788
CV est 6.913817894543462
returned est: 6.913817894543462 returned var 77.44523976636054


In [85]:
s_geo = np.zeros(trials)

for i in range(trials ):
    test = GBM_Euler(T, K, S, sigma, r, M)
    s_geo[i] = np.max((0, geo_mean_overflow(test)   - K))

V_MC_geo = np.mean(s_geo)
var_geo = np.var(s_geo)
CI = [np.mean(s_geo) - 1.96*np.std(s_geo)/np.sqrt(len(s_geo)), np.mean(s_geo) + 1.96*np.std(s_geo)/np.sqrt(len(s_geo))]

print(f"Asian MC geometric: {V_MC_geo}")
print(f"Asian MC geometric var: {var_geo}")

    
"""
Monte Carlo for Arithmetic Average of Asian Call option
"""
s_ar = np.zeros(trials)

for i in range(trials):
    test = GBM_Euler(T, K, S, sigma, r, M)
    S_arith = ((1/len(test)) * np.sum(test))
    s_ar[i] = np.max((0, S_arith - K))

V_MC_arith = np.mean(s_ar)
var_arith = np.var(s_ar)
CI = [np.mean(s_ar) - 1.96*np.std(s_ar)/np.sqrt(len(s_ar)), np.mean(s_ar) + 1.96*np.std(s_ar)/np.sqrt(len(s_ar))]

# Compute optimal beta coefficient which minimizes the variance
beta = (np.std(s_ar) / np.std(s_geo)) * np.corrcoef(s_ar, s_geo)[0][0]

# Compute Control Variate estimate of arithmetic Asian call option
arith_CV_est = V_MC_arith - beta*(V_MC_geo - True_Asian_geo)

var_CV_arith = var_arith - ((np.cov((s_ar, s_geo))[0][0]**2) / var_geo)

print(f"Asian MC arith: {V_MC_arith}")
print(f"Asian MC arith var: {var_arith}")
print(f"beta: {beta}")
print(f"CV est {arith_CV_est}")
print(f"CV var {var_CV_arith}")

Asian MC geometric: 3.1333560623298324
Asian MC geometric var: 16.02195857922662
Asian MC arith: 3.361307249249232
Asian MC arith var: 17.780563942267428
beta: 1.0534525126090395
CV est 3.1848551600761494
CV var -1.9911574211577943


In [86]:
"""
Monte Carlo for Arithmetic Average of Asian Call option
"""
s_ar = np.zeros(trials)

for i in range(trials):
    test = GBM_Euler(T, K, S, sigma, r, M)
    S_arith = ((1/len(test)) * np.sum(test))
    s_ar[i] = np.max((0, S_arith - K))

V_MC_arith = np.mean(s_ar)
var_arith = np.var(s_ar)
CI = [np.mean(s_ar) - 1.96*np.std(s_ar)/np.sqrt(len(s_ar)), np.mean(s_ar) + 1.96*np.std(s_ar)/np.sqrt(len(s_ar))]


print(f"Asian MC arith: {V_MC_arith}")
print(f"Asian MC arith var: {var_arith}")

#print(f"Confidence interval: {CI[0], CI[1]}")
#print(f"Size of CI: {CI[1]-CI[0]}")

Asian MC arith: 3.359608767066635
Asian MC arith var: 17.618755206212438


In [87]:
# Compute optimal beta coefficient which minimizes the variance
beta = (np.std(s_ar) / np.std(s_geo)) * np.corrcoef(s_ar, s_geo)[0][0]

# Compute Control Variate estimate of arithmetic Asian call option
arith_CV_est = V_MC_arith - beta*(V_MC_geo - True_Asian_geo)
print(arith_CV_est)

3.18396139769211


In [88]:
# Variance of controlled estimator (arith_CV_est)
# var_CV_arith = (1/len(test)) * (var_geo - (np.cov((s_ar, s_geo))[0][0]**2 / var_arith))
var_CV_arith = var_arith - ((np.cov((s_ar, s_geo))[0][0]**2) / var_geo)
print(var_CV_arith)

-1.7947457745350697
